In [1]:
# Import the Earth Engine Python Package
import ee

# Import Dependencies
import pandas as pd
import matplotlib.dates as mdates
from IPython.display import Image
from datetime import datetime
from matplotlib import dates
from pylab import *

ee.Initialize()

In [5]:
# build geometry to measure
rect = ee.Geometry.Polygon(
        [[[106.81591544702997, -6.33681462466857],
          [106.81591544702997, -6.350036929099756],
          [106.82780299738397, -6.350036929099756],
          [106.82780299738397, -6.33681462466857]]])

In [11]:
# convert geometry to JSON format
rect_JSON = rect.getInfo()['coordinates']

In [12]:
# set variables
SCALE = 10;
polarization = 'VV'; # vertical/vertical or vertical/horizontal

In [13]:
# initiate google earth engine to aquire the image
S1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
.filterDate('2016-01-01', '2019-12-12')\
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))\
.filter(ee.Filter.eq('instrumentMode', 'IW'))\
.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
.select(polarization)\
.filterBounds(rect);

# interactive plotting using folium (leaflet wrapper)

In [14]:
import folium
print(folium.__version__)


# iets
palette = ['0784b5', '39ace7', '9bd4e4', 'cadeef', 'ffffff']

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

0.10.0


In [15]:
# water function:
def waterfunction(image):
  
    image2 = image  .reproject(crs ='EPSG:4326', scale = SCALE)\
                      .focal_mode()\
                      .focal_max(3).focal_min(5).focal_max(3)\
                      .reproject(crs ='EPSG:4326', scale = SCALE)
  
    # get pixels above the threshold
    water01 = image2.lt(-12)
    # mask those pixels from the image
    image = image.updateMask(water01)
    #  calculate pixelalreas
    area = ee.Image.pixelArea()
    waterArea = water01.multiply(area).rename('waterArea');
    #   add waterarea band to uitput
    image = image.addBands(waterArea)

    return image


#  run waterfunction over collection of Sentinel
collection = S1.map(waterfunction)


In [16]:
# Use folium to visualize the imagery.
# mapid = S1.first().getMapId({
#     'region':rect_JSON,
#     'min':-25,
#     'max':0,
#     'palette':['0784b5', '39ace7', '9bd4e4', 'cadeef', 'ffffff']
# })

mapid = collection.select('waterArea').first().getMapId({
    'region':rect_JSON,
    'min':0,
    'max':1,
    'palette':['ffffff','0784b5']
})


map = folium.Map(location=[-6.36456378799,106.812382228])
folium.TileLayer(
    tiles = EE_TILES.format(**mapid),
    attr = 'Google Earth Engine',
    overlay = True,
    name ='median composite',
    
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
# if you want to save the map to html widget:
map.save(outfile='test.html')